# exploration 16 Session based Recommendation 시스템 제작  
***
### 데이터 셋: Movielens 1M Dataset 


In [1]:
import tensorflow as tf
import os
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Loading Data

In [2]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


### Preprocess

- Time 항목에는 UTC time 가 포함되어, 1970년 1월 1일부터 경과된 초단위 시간이 기재되어 datetime 형식으로 바꿈.

In [3]:
#Time 데이터(Second)를 Datetime으로 바꾸기
import datetime as dt
from datetime import date
from datetime import timedelta

start = '1970-01-01 00:00:00.000000'
start = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S.%f') #start:1970-01-01 00:00:00

date = []
for delta in data['Time'] :
    date.append(start + timedelta(seconds = delta))
    
data['Time'] = date
data.head()

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43


In [4]:
#데이터의 전체적인 통계확인 
data.iloc[:, :-1].describe().T.sort_values(by='std' , ascending = False)\
                     .style.background_gradient(cmap='GnBu')\
                     .bar(subset=["max"], color='#F8766D')\
                     .bar(subset=["mean",], color='#00BFC4')

,count,mean,std,min,25%,50%,75%,max
UserId,1000209.000000,3024.512348,1728.412695,1.000000,1506.000000,3070.000000,4476.000000,6040.000000
ItemId,1000209.000000,1865.539898,1096.040689,1.000000,1030.000000,1835.000000,2770.000000,3952.000000
Rating,1000209.000000,3.581564,1.117102,1.000000,3.000000,4.000000,4.000000,5.000000


In [5]:
data['UserId'].nunique(), data['ItemId'].nunique() #유저수와 아이템수 확인 

(6040, 3706)

In [6]:
user_length = data.groupby('UserId').size() #유저 Id의 session length 확인
user_length #동일한 userId를 공유하는 데이터 row 개수 

UserId
1        53
2       129
3        51
4        21
5       198
       ... 
6036    888
6037    202
6038     20
6039    123
6040    341
Length: 6040, dtype: int64

In [7]:
print(user_length.describe().T)
print("=======================================================")
print("user_length median: {}, 99.9%: {}".format(user_length.median(), user_length.quantile(0.999)))   

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
dtype: float64
user_length median: 96.0, 99.9%: 1343.181000000005


- user_id 길이 중앙값 96.0, 한명의 유저가 보통 96개 영화를 시청함/96개의 영화에 대한 평점을 남김 
- 6040명의 유저 
- 99.9% 1343 요건 어떻게 해석해야할지 아직 모르겠음, 99.9% 유저가 1343개 이하의 영화를 시청했다는 뜻? 

- 위의 결과가 납득이 안되서 검색해보고 이전 기수 분의 코드를 참고하여 userid와 time을 기준으로 새로운 데이터 프레임을 합침 
- [code reference](https://github.com/YOOHYOJEONG/AIFFEL_LMS_project/blob/master/ex12/ex12_Session_Based_Recommendation.ipynb)

In [8]:
#groupby를 이용하여 UserId와 Time을 기준으로 새로운 데이터 프레임을 생성
user_time = data.groupby(['UserId', 'Time'])['ItemId'].count().reset_index()
user_time.reset_index(inplace = True)
user_time.head()

,index,UserId,Time,ItemId
0,0,1,2000-12-31 22:00:19,1
1,1,1,2000-12-31 22:00:55,3
2,2,1,2000-12-31 22:01:43,1
3,3,1,2000-12-31 22:02:52,1
4,4,1,2000-12-31 22:04:35,1


In [9]:
#UserId와 Time을 기준으로 merge
new_data = pd.merge(data, user_time, on = ['UserId', 'Time'])
new_data

,UserId,ItemId_x,Rating,Time,index,ItemId_y
0,1,3186,4,2000-12-31 22:00:19,0,1
1,1,1270,5,2000-12-31 22:00:55,1,3
2,1,1721,4,2000-12-31 22:00:55,1,3
3,1,1022,5,2000-12-31 22:00:55,1,3
4,1,2340,3,2000-12-31 22:01:43,2,1
...,...,...,...,...,...,...
1000204,6040,2917,4,2001-08-10 14:40:29,471159,1
1000205,6040,1921,4,2001-08-10 14:41:04,471160,2
1000206,6040,1784,3,2001-08-10 14:41:04,471160,2
1000207,6040,161,3,2001-08-10 14:41:26,471161,1


- ItemId_y 칼럼 삭제
- userid와 time이 하나의 session으로 취급 
- index_x를 sessionid로 변경

In [10]:
#불필요한 칼럼 삭제
new_data.drop(columns = 'ItemId_y', inplace = True)

#칼럼 명 수정
new_data.rename(columns = {'ItemId_x' : 'ItemId'}, inplace = True)
new_data.rename(columns = {'index' : 'SessionId'}, inplace = True)

new_data

,UserId,ItemId,Rating,Time,SessionId
0,1,3186,4,2000-12-31 22:00:19,0
1,1,1270,5,2000-12-31 22:00:55,1
2,1,1721,4,2000-12-31 22:00:55,1
3,1,1022,5,2000-12-31 22:00:55,1
4,1,2340,3,2000-12-31 22:01:43,2
...,...,...,...,...,...
1000204,6040,2917,4,2001-08-10 14:40:29,471159
1000205,6040,1921,4,2001-08-10 14:41:04,471160
1000206,6040,1784,3,2001-08-10 14:41:04,471160
1000207,6040,161,3,2001-08-10 14:41:26,471161


In [11]:
#session length 확인
session_length = new_data.groupby('SessionId').size()
print(len(session_length))

471163


In [12]:
print(session_length.describe().T)
print("=======================================================")
print("session_length median: {}, 99.9%: {}".format(session_length.median(), session_length.quantile(0.999)))   

count    471163.000000
mean          2.122851
std           1.546899
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max          30.000000
dtype: float64
session_length median: 2.0, 99.9%: 10.0


- 중앙값과 평균을 바탕으로 세션 하나당 약 2개의 영화를 평가 
- 최대 30개 최소 1개 영화를 평가
- 99.9&의 세션이 10개 이하의 영화를 평가 

In [13]:
#30개 영화 평가한 세션 확인 
long_session = session_length[session_length==30].index[0]
display(new_data[new_data['SessionId']==long_session])
new_data[new_data['SessionId']==long_session].shape

,UserId,ItemId,Rating,Time,SessionId
112347,731,3044,4,2000-11-29 20:06:42,55117
112348,731,1455,3,2000-11-29 20:06:42,55117
112349,731,1639,5,2000-11-29 20:06:42,55117
112350,731,3244,4,2000-11-29 20:06:42,55117
112351,731,1656,2,2000-11-29 20:06:42,55117
112352,731,3426,4,2000-11-29 20:06:42,55117
112353,731,1829,2,2000-11-29 20:06:42,55117
112354,731,2675,4,2000-11-29 20:06:42,55117
112355,731,802,3,2000-11-29 20:06:42,55117
112356,731,803,5,2000-11-29 20:06:42,55117


(30, 5)

- 2000-11-29 20:06:42, 1초 안에 30개 영화를 평가했다는 것은 이상치로 판단함 

In [14]:
#이상치 제거
new_data = new_data.loc[new_data['SessionId'] != long_session]
new_data

,UserId,ItemId,Rating,Time,SessionId
0,1,3186,4,2000-12-31 22:00:19,0
1,1,1270,5,2000-12-31 22:00:55,1
2,1,1721,4,2000-12-31 22:00:55,1
3,1,1022,5,2000-12-31 22:00:55,1
4,1,2340,3,2000-12-31 22:01:43,2
...,...,...,...,...,...
1000204,6040,2917,4,2001-08-10 14:40:29,471159
1000205,6040,1921,4,2001-08-10 14:41:04,471160
1000206,6040,1784,3,2001-08-10 14:41:04,471160
1000207,6040,161,3,2001-08-10 14:41:26,471161


In [15]:
new_data['Time'].min(), new_data['Time'].max()

(Timestamp('2000-04-25 23:05:32'), Timestamp('2003-02-28 17:49:50'))

In [16]:
time2000 = new_data[new_data['Time'] < dt.datetime(2001,1,1)]#2000년 데이터
time2001 = new_data[(new_data['Time'] > dt.datetime(2000,12,31)) & (new_data['Time'] < dt.datetime(2002,1,1))]#2001년 데이터 
time2002 = new_data[(new_data['Time'] >= dt.datetime(2002,1,1)) & (new_data['Time'] < dt.datetime(2003,1,1))]#2002년 데이터 
time2003 = new_data[new_data['Time'] > dt.datetime(2002,12,31)] #2003년 데이터 
print("2000년 데이터 개수: {}".format(time2000.shape[0]))
print("2001년 데이터 개수: {}".format(time2001.shape[0]))
print("2002년 데이터 개수: {}".format(time2002.shape[0]))
print("2003년 데이터 개수: {}".format(time2003.shape[0]))

2000년 데이터 개수: 904727
2001년 데이터 개수: 70230
2002년 데이터 개수: 24046
2003년 데이터 개수: 3369


- 2000년에서 2003년으로 갈수록 데이터 적어짐 

In [17]:
#평점 3점 이상인 영화만 남기고 제거 
new_data = new_data[new_data['Rating'] >= 3]

### Dataset 분리 
- test set: last_time에서 100일 전까지
- train set: 100일 제외한 나머지 기간 
- validation: train set의 last time에서 365일 전까지 

In [18]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    session_last_time = data.groupby('SessionId')['Time'].max()
    session_in_train = session_last_time[session_last_time < final_time - dt.timedelta(n_days)].index
    session_in_test = session_last_time[session_last_time >= final_time - dt.timedelta(n_days)].index

    before_date = data[data['SessionId'].isin(session_in_train)]
    after_date = data[data['SessionId'].isin(session_in_test)]
    after_date = after_date[after_date['ItemId'].isin(before_date['ItemId'])]
    return before_date, after_date

In [19]:
#test dataset 분리
train, test = split_by_date(new_data, n_days = 100)
#validation dataset 분리
train, val = split_by_date(train, n_days = 365)

In [20]:
#new_data에 대한 정보.
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["SessionId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

In [21]:
stats_info(train, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 810327
	 Sessions: 404871
	 Items: 3612
	 First Time : 2000-04-25 23:05:32
	 Last Time : 2001-11-20 05:13:09

* valid Set Stats Info
	 Events: 21991
	 Sessions: 15450
	 Items: 2820
	 First Time : 2001-11-20 19:04:49
	 Last Time : 2002-11-20 16:38:40

* test Set Stats Info
	 Events: 4118
	 Sessions: 3071
	 Items: 1625
	 First Time : 2002-11-20 20:30:02
	 Last Time : 2003-02-28 17:49:50



In [22]:
#train data를 기준으로 인덱싱.
id2idx = {item_id : index for index, item_id in enumerate(train['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  #id2idx에 없는 아이템은 모르는 값(-1) 처리.
    return df

train = indexing(train, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [23]:
#전처리 완료 된 데이터 저장.
save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

train.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

### Data Pipeline

In [24]:
#데이터가 주어지면 세션이 시작되는 인덱스를 담는 값과 세션을 새로 인덱싱한 값을 갖는 클래스
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['SessionId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['SessionId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('SessionId').size().cumsum()
        return offsets

In [25]:
#train데이터로 SessionDataset 객체를 만들기.
train_dataset = SessionDataset(train)
train_dataset.df.head(2)

,UserId,ItemId,Rating,Time,SessionId,item_idx
0,1,3186,4,2000-12-31 22:00:19,0,0
1,1,1270,5,2000-12-31 22:00:55,1,1


In [26]:
train_dataset.click_offsets #click_offsets : 각 세션이 시작된 인덱스 담고 있음.
train_dataset.session_idx #각 세션을 인덱싱한 np.array

array([     0,      1,      2, ..., 404868, 404869, 404870])

### SessionDataLoader
- SessionDataLoader:SessionDataset 객체를 받아서 Session-Parallel mini-batch를 만드는 클래스  
- __iter__ 메서드는 모델 인풋, 라벨, 세션이 끝나는 곳의 위치를 yield함
- mask는 후에 RNN Cell State를 초기화하는데 사용할 것

In [27]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옴
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장함
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져오기
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져오기
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것임
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update함
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줌
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻 mask에 기록

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록
        return start, end, mask, last_session, finished

In [28]:
train_data_loader = SessionDataLoader(train_dataset, batch_size=4)
iter_ex = iter(train_data_loader)
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [19  1  7  9]
Label Item Idx are :       [20  2  8 10]
Previous Masked Input Idx are [0]


### Modeling 
- Evaluation Metric
MRR과 Recall@k를 사용할 것.
MRR은 정답 아이템이 나온 순번의 역수 값으로 정답 아이템이 추천 결과 앞쪽 순번에 나온다면 지표가 높아질 것이고 뒤쪽에 나오거나 안나온다면 지표가 낮아질 것.

In [29]:
#Evaluation Metric

def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

In [30]:
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [31]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=RMSprop(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [37]:
#모델에 사용할 hyper-parameter를 class형식으로 관리

class Args:
    def __init__(self, train, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.train = train
        self.val = val
        self.test = test
        self.num_items = train['ItemId'].nunique()
        self.num_sessions = train['SessionId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

#args = Args(train, val, test, batch_size=128, hsz=50, drop_rate=0.1, lr=0.001, epochs=15, k=20)
args = Args(train, val, test, batch_size=256, hsz=50, drop_rate=0.1, lr=0.0001, epochs=20, k=20)

### MRR의 수치가 올라갈수록 추천 결과가 정답 결과와 가깝게 나왔다는 것 
### case1.batch_size=128, hsz=50, drop_rate=0.1, lr=0.001, epochs=15, k=20
- Recall@20: 0.220982
- MRR@20: 0.086334
  
### case2.batch_size=256, hsz=50, drop_rate=0.1, lr=0.001, epochs=15, k=20
- Recall@20: 0.227865
- MRR@20: 0.100683
       
### case3.batch_size=512, hsz=50, drop_rate=0.1, lr=0.001, epochs=15, k=20
- Recall@20: 0.236328
- MRR@20: 0.073218
  
### case4.batch_size=256, hsz=50, drop_rate=0.1, lr=0.001, epochs=20, k=20
- Recall@20: 0.246094
- MRR@20: 0.094746
  
### case5.batch_size=256, hsz=50, drop_rate=0.1, lr=0.0001, epochs=20, k=20
- Recall@20: 0.143229
- MRR@20: 0.051592    


In [38]:
model = create_model(args)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(256, 1, 3612)]          0         
_________________________________________________________________
GRU (GRU)                    [(256, 50), (256, 50)]    549600    
_________________________________________________________________
dropout_1 (Dropout)          (256, 50)                 0         
_________________________________________________________________
dense_1 (Dense)              (256, 3612)               184212    
Total params: 733,812
Trainable params: 733,812
Non-trainable params: 0
_________________________________________________________________


### model 훈련 

In [39]:
#train 셋으로 학습하면서 valid 셋으로 검증.
def train_model(model, args):
    train_dataset = SessionDataset(args.train)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.train) - args.train['SessionId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  #종료된 session은 hidden_state를 초기화. 아래 메서드에서 확인할 수 있음.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  #valid set에 대해 검증.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")

def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  #model에서 gru layer를 가져옴.
    hidden_states = gru_layer.states[0].numpy()  #gru_layer의 parameter를 가져옴.
    for elt in mask:  #mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  #parameter를 초기화 함.
    gru_layer.reset_states(states=hidden_states)

#valid셋과 test셋을 평가하는 코드
def get_metrics(data, model, args, k: int): 
    
    #train과 거의 같지만 mrr, recall을 구하는 라인이 있음.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['SessionId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  #softmax 값이 큰 순서대로 sorting.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [40]:
train_model(model, args)

Evaluation:  96%|█████████▌| 24/25 [00:26<00:01,  1.11s/it]


	 - Recall@20 epoch 1: 0.048665
	 - MRR@20    epoch 1: 0.010453



Evaluation:  96%|█████████▌| 24/25 [00:26<00:01,  1.10s/it]


	 - Recall@20 epoch 2: 0.048503
	 - MRR@20    epoch 2: 0.010432



Evaluation:  96%|█████████▌| 24/25 [00:26<00:01,  1.10s/it]


...

	 - Recall@20 epoch 18: 0.136882
	 - MRR@20    epoch 18: 0.040386



Evaluation:  96%|█████████▌| 24/25 [00:25<00:01,  1.04s/it]


	 - Recall@20 epoch 19: 0.140788
	 - MRR@20    epoch 19: 0.042779



Evaluation:  96%|█████████▌| 24/25 [00:25<00:01,  1.04s/it]

	 - Recall@20 epoch 20: 0.146322
	 - MRR@20    epoch 20: 0.044513



### 최종 성능 확인

In [41]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)

Evaluation:  75%|███████▌  | 3/4 [00:03<00:01,  1.04s/it]

	 - Recall@20: 0.143229
	 - MRR@20: 0.051592



### Discussion  
- movielens 데이터에 장르와 영화 제목, 개봉년도도 있었던 걸로 기억하는데 다음번엔 이런 feature도 함께 사용해 봐야겠다.  
- 아직 session baed recommendation에 대한 이해가 부족해 더 깊게 공부해야겠다.  
- batch_size=256, hsz=50, drop_rate=0.1, lr=0.001, epochs=20, k=20 일 때 가장 결과가 좋았다. 
- Recall@20: 0.246094
- MRR@20: 0.094746